In [1]:
import numpy as np
import pandas as pd
import math
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({'font.size': 8})

In [ ]:
from sklearn.datasets import load_digits

def digitize_catg_feature(col_index, df):
    ladels = pd.factorize(df.iloc[:, col_index].unique(), na_sentinel=None)
    ladels_replacement = {}
    for i in range(len(ladels[0])): ladels_replacement[ladels[1][i]] = ladels[0][i]
    df.iloc[:, col_index] = df.iloc[:, col_index].replace(ladels_replacement)
    return df

# Wdbc
# data = pd.read_csv(f'datasets/DS01.data', header=None)
# data = data.replace('?', np.nan)
# data = digitize_catg_feature(1, data)
# data = data.dropna()
# data.insert(len(data.columns)-1, 'LABEL', data.pop(1))
# data.to_csv('datasets/DS01.csv', index=False)

# Lung canser
# data = pd.read_csv(f'datasets/DS02.data', header=None)
# data = data.replace('?', np.nan)
# data = data.dropna()
# data.insert(len(data.columns)-1, 'LABEL', data.pop(0))
# data.to_csv('datasets/DS02.csv', index=False)

# data = pd.read_csv(f'datasets/DS03.data', header=None)
# data = data.replace('?', np.nan)
# data = digitize_catg_feature(-1, data)
# data = data.dropna()
# data.insert(len(data.columns)-1, 'LABEL', data.pop(data.columns[-1]))
# data.to_csv('datasets/DS03.csv', index=False)


# digits = load_digits()
# data = pd.DataFrame(data=pd.DataFrame(data=digits['data']))
# data.insert(len(data.columns), 'LABEL', digits['target'])
# data = data.sample(n=1000)
# data.to_csv('datasets/DS04.csv', index=False)

# data = pd.read_csv(f'datasets/raw/MADELON/madelon_train.data', delimiter=r"\s+", header=None)
# y = pd.read_csv(f'datasets/raw/MADELON/madelon_train.labels', header=None)
# data['LABEL'] = y[0]
# data = data.sample(n=600)
# data.to_csv('datasets/DS05.csv', index=False)

# data = pd.read_csv(f'datasets/DS06.data')
# data.drop(columns=data.columns[-5:], axis=1, inplace=True)
# data.drop(columns=["FLOOR"], inplace=True)
# data.insert(len(data.columns)-1, 'LABEL', data.pop(data.columns[-1]))
# data = data.sample(n=900)
# data.to_csv('datasets/DS06.csv', index=False)

# data = pd.read_csv(f'datasets/X_train.txt', delimiter=r"\s+", header=None)
# y = pd.read_csv(f'datasets/y_train.txt', header=None)
# data['LABEL'] = y[0]
# data = data.sample(n=900)
# data.to_csv('datasets/DS07.csv', index=False)

# data = pd.read_csv(f'datasets/final_X_train.txt', delimiter=",", header=None)
# y = pd.read_csv(f'datasets/final_y_train.txt', header=None)
# data['LABEL'] = y[0]
# data = data.sample(n=900)
# data.to_csv('datasets/DS07-2.csv', index=False)

# HAPT
# data = pd.read_csv(f'datasets/raw/HAPT/X_train.txt', delimiter=r"\s+", header=None)
# y = pd.read_csv(f'datasets/raw/HAPT/y_train.txt', header=None)
# data['LABEL'] = y[0]
# data = data.sample(n=1200)
# data.to_csv('datasets/DS08.csv', index=False)

# isolet5
# data = pd.read_csv(f'datasets/isolet5.data', header=None)
# data.insert(len(data.columns)-1, 'LABEL', data.pop(data.columns[-1]))
# data = data.sample(n=1040)
# data.to_csv('datasets/DS09.csv', index=False)


# fac = pd.read_csv(f'datasets/raw/mfeat/mfeat-fac', delimiter=r"\s+", header=None)
# fou = pd.read_csv(f'datasets/raw/mfeat/mfeat-fou', delimiter=r"\s+", header=None)
# kar = pd.read_csv(f'datasets/raw/mfeat/mfeat-kar', delimiter=r"\s+", header=None)
# mor = pd.read_csv(f'datasets/raw/mfeat/mfeat-mor', delimiter=r"\s+", header=None)
# pix = pd.read_csv(f'datasets/raw/mfeat/mfeat-pix', delimiter=r"\s+", header=None)
# zer = pd.read_csv(f'datasets/raw/mfeat/mfeat-zer', delimiter=r"\s+", header=None)
# data = pd.concat([fac, fou, kar, mor, pix, zer], axis=1)
# data['LABEL']=np.divmod(np.arange(len(data)), 200)[0]
# data = data.sample(n=1000)
# data.to_csv('datasets/DS10.csv', index=False)

: 

In [ ]:
class MOBGA_AOS:
    def __init__(self, LP = 5, max_nFE = 500, pop_size = 100, p_c = 0.9):
        self.max_nFE = max_nFE
        self.pop_size = pop_size
        self.operator_pool = ['single-point', 'two-point', 'uniform', 'shuffle', 'reduced-surrogate']
        self.LP = LP
        self.p_c = p_c

        # Q = pool_size
        self.Q = len(self.operator_pool)
        self.OSP = [1/self.Q for _ in range(self.Q)]

    def load_data(self, dataset_name):
        data = pd.read_csv(f'datasets/{dataset_name}').to_numpy()
        self.X = data[:, :-1]
        self.y = data[:, -1]
        self.d = self.X.shape[1]
        self.p_m = 1/self.d
        # self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.4, random_state=0)

    def init_pop(self):
        if(self.d > 250):
            pop = []
            for _ in range(self.pop_size):
                individual = np.zeros(self.d)
                individual[:100] = 1
                np.random.shuffle(individual)
                pop.append(individual)
        else:
            return [np.random.choice([0, 1], size=self.d) for _ in range(self.pop_size)]
    
    def one_point_crossover(self, p1, p2):
        cross_point = np.random.randint(0, self.d)
        c1 = list(p1[:cross_point]) + list(p2[cross_point:])
        c2 = list(p2[:cross_point]) + list(p1[cross_point:])
        return [c1, c2]
    
    def two_point_crossover(self, p1, p2):
        cross_points = np.random.randint(0, self.d, 2)
        cross_points = np.sort(cross_points)
        c1 = list(p1[:cross_points[0]]) + list(p2[cross_points[0]:cross_points[1]]) + list(p1[cross_points[1]:])
        c2 = list(p2[:cross_points[0]]) + list(p1[cross_points[0]:cross_points[1]]) + list(p2[cross_points[1]:])
        return [c1, c2]

    def uniform_crossover(self, p1, p2):
        c1 = np.full(self.d, None).tolist()
        c2 = np.full(self.d, None).tolist()
        for i in range(self.d):
            rand_rate = np.random.uniform(0, 1)
            if(rand_rate >= 0.5):
                c1[i] = p1[i]
                c2[i] = p2[i]
            else:
                c1[i] = p2[i]
                c2[i] = p1[i]
        return [c1, c2]
    
    def shuffle_crossover(self, p1, p2):
        c1, c2 = p1, p2
        index = np.random.choice(np.arange(0, self.d), self.d, replace = False)
        ran = np.random.randint(0, self.d)
        for i in range(ran):
            ind = index[i]
            c1[ind] = p2[ind]
            c2[ind] = p1[ind]
        return [c1, c2]
    
    # def shuffle_crossover(self, p1, p2):
    #     p1 = shuffle(p1)
    #     p2 = shuffle(p2)
    #     offsrping = self.one_point_crossover(p1, p2)
    #     return offsrping

    def reduced_surrogate_crossover(self, p1, p2):
        if(len(np.array(np.where(np.array(p1) != np.array(p2))).flatten()) > 0):
            cross_point = np.random.choice(np.array(np.where(np.array(p1) != np.array(p2))).flatten())
            c1 = list(p1[:cross_point]) + list(p2[cross_point:])
            c2 = list(p2[:cross_point]) + list(p1[cross_point:])
            return [c1, c2]
        else:
            return [p1, p2]
    
    def crossover(self, parents, operator_idx):
        p = np.random.uniform(0, 1)
        if p > self.p_c:
            return parents
        
        op_name = self.operator_pool[operator_idx]
        if(op_name == "single-point"):
            offspring = self.one_point_crossover(parents[0], parents[1])
        elif(op_name == "two-point"):
            offspring = self.two_point_crossover(parents[0], parents[1])
        elif(op_name == "uniform"):
            offspring = self.uniform_crossover(parents[0], parents[1])
        elif(op_name == "shuffle"):
            offspring = self.shuffle_crossover(parents[0], parents[1])
        elif(op_name == "reduced-surrogate"):
            offspring = self.reduced_surrogate_crossover(parents[0], parents[1])
        else:
            raise Exception("Unknown crossover operator!")
        
        for i, child in enumerate(offspring):
            if(not np.any(child)): 
                print(op_name)
                print("XOVER")
                print(f"C: {offspring[i]}")
                offspring[i] = parents[i]
                print(f"P: {parents[i]}")
        
        return offspring
    
    def mutation(self, individual):
        # Select a fraction of individual
        frac_idxs = np.random.choice(range(self.d), int(0.5 * self.d))
        indiv = individual.copy()
        for idx in frac_idxs:
            p = np.random.uniform(0, 1)
            if p < self.p_m:
                new_gene = np.random.randint(0, 2)
                # indiv[idx] = new_gene
                indiv[idx] = 0 if (indiv[idx] == 1) else 1
        if(not np.any(indiv)): 
            print("MUTE")
            print(f"C: {individual}")
            print(f"P: {indiv}")
            idx = np.random.randint(0, self.d)
            individual[idx] = 1
            return individual
        else:
            return indiv

    def score_individual_1(self, individual):
        model = KNeighborsClassifier(n_neighbors=3)
        indexes = np.array(np.where(np.array(individual) == 1)).flatten()
        X = (self.X[:, indexes]).reshape(len(self.X), len(indexes))
        ss = ShuffleSplit(n_splits=3, test_size=0.3, random_state=10)
        Nerr_Nall = []
        for train_index, test_index in ss.split(X):
            model.fit(X[train_index], self.y[train_index])
            y_pred = model.predict(X[test_index])
            y_test = self.y[test_index]
            # misclassified = np.where(y_test != y_pred)
            Nerr_Nall.append((np.sum(y_pred != y_test))/len(y_test))
        return np.round(np.mean(Nerr_Nall) * 100, 2)

    def score_individual_2(self, individual):
        return np.sum(individual)

    def fast_non_dominated_sort(self, values1, values2):
        S = [[] for i in range(0,len(values1))]
        front = [[]]
        n = [0 for i in range(0,len(values1))]
        rank = [0 for i in range(0, len(values1))]

        for p in range(0, len(values1)):
            S[p] = []
            n[p] = 0
            for q in range(0, len(values1)):
                if (values1[p] < values1[q] and values2[p] < values2[q]) \
                    or (values1[p] <= values1[q] and values2[p] < values2[q]) \
                    or (values1[p] < values1[q] and values2[p] <= values2[q]):
                    if q not in S[p]:
                        S[p].append(q)
                elif (values1[q] < values1[p] and values2[q] < values2[p]) \
                    or (values1[q] <= values1[p] and values2[q] < values2[p]) \
                    or (values1[q] < values1[p] and values2[q] <= values2[p]):
                    n[p] = n[p] + 1
            if n[p] == 0:
                rank[p] = 0
                if p not in front[0]:
                    front[0].append(p)

        i = 0
        while(front[i] != []):
            Q = []
            for p in front[i]:
                for q in S[p]:
                    n[q] = n[q] - 1
                    if(n[q] == 0):
                        rank[q] = i+1
                        if q not in Q:
                            Q.append(q)
            i = i+1
            front.append(Q)

        del front[len(front)-1]
        return front
    
    def index_locator(self, a, list):
        for i in range(0, len(list)):
            if list[i] == a:
                return i
        return -1

    def sort_by_values(self, list1, values):
        sorted_list = []
        while(len(sorted_list)!=len(list1)):
            if self.index_locator(min(values),values) in list1:
                sorted_list.append(self.index_locator(min(values),values))
            values[self.index_locator(min(values),values)] = math.inf
        return sorted_list

    def crowding_distance(self, values1, values2, front):
        distance = [0 for i in range(0,len(front))]
        sorted1 = self.sort_by_values(front, values1[:])
        sorted2 = self.sort_by_values(front, values2[:])
        distance[0] = 9999999999999999
        distance[len(front) - 1] = 9999999999999999
        for k in range(1,len(front)-1):
            distance[k] = distance[k] + (values1[sorted1[k+1]] - values2[sorted1[k-1]])/(max(values1)-min(values1))
        for k in range(1,len(front)-1):
            distance[k] = distance[k] + (values1[sorted2[k+1]] - values2[sorted2[k-1]])/(max(values2)-min(values2))
        return distance
        
    def update_OSP(self, RD, PN):
        S_q_1 = RD.sum(axis=0)
        S_q_2 = PN.sum(axis=0)

        S_q_3 = S_q_2
        S_q_3[S_q_3 == 0] = 0.0001

        S_q_4 = S_q_1/(S_q_2 + S_q_3)
        self.OSP = S_q_4/np.sum(S_q_4)
        if(np.isnan(self.OSP).all()): 
            print("OSP FUCKED")
            self.OSP = [1/self.Q for _ in range(self.Q)]

    # def dominance_comparison(self, individuals):
    #     objective1_values = [self.score_individual_1(individual) for individual in individuals]
    #     objective2_values = [self.score_individual_2(individual) for individual in individuals]
    #     return self.fast_non_dominated_sort(objective1_values, objective2_values)

    # def credit_assigment(self, parents, offspring, op_idx):
    #     paretos = self.dominance_comparison(offspring)

    #     if(len(paretos[0]) == 1):
    #         for i in range(2):
    #             individuals = [parents[paretos[0][0]], offspring[i]]
    #             comparisions = self.dominance_comparison(individuals)
    #             if(len(comparisions[0]) == 1 and comparisions[0][0] == 1):
    #                 self.n_penalty[op_idx] += 1
    #             else:
    #                 self.n_reward[op_idx] += 1
    #     else:
    #         for i in range(2):
    #             comparision_individuals = [parents[paretos[0][0]], offspring[i]]
    #             comparisions1 = self.dominance_comparison(comparision_individuals)
    #             comparision_individuals = [parents[paretos[0][1]], offspring[i]]
    #             comparisions2 = self.dominance_comparison(comparision_individuals)
    #             if((len(comparisions1[0]) != 1) and (len(comparisions2[0]) != 1)\
    #                or (((len(comparisions1[0]) == 1) and comparisions1[0][0] != 1)\
    #                 and ((len(comparisions2[0]) == 1) and comparisions2[0][0] != 1))):
    #                 self.n_reward[op_idx] += 1
    #             else:
    #                 self.n_penalty[op_idx] += 1

    # def dominance_comparison(self, individuals):
    #     objective1_values = [self.score_individual_1(individual) for individual in individuals]
    #     objective2_values = [self.score_individual_2(individual) for individual in individuals]
    #     and_condition = True
    #     or_condition = False
    #     for first, second in zip(objective1_values, objective2_values):
    #         and_condition = and_condition and first <= second
    #         or_condition = or_condition or first < second
    #     return (and_condition and or_condition)

    # def dominance_comparison(self, individuals):
    #     objective1_values = [self.score_individual_1(individual) for individual in individuals]
    #     objective2_values = [self.score_individual_2(individual) for individual in individuals]
    #     return ((objective1_values[0] < objective1_values[1]) and (objective2_values[0] < objective2_values[1]))

    def dominance_comparison(self, individuals):
        objective1_values = [self.score_individual_1(individual) for individual in individuals]
        objective2_values = [self.score_individual_2(individual) for individual in individuals]
        and_cond = ((objective1_values[0] <= objective1_values[1]) and (objective2_values[0] <= objective2_values[1]))
        or_cond = ((objective1_values[0] < objective1_values[1]) or (objective2_values[0] < objective2_values[1]))
        return (and_cond and or_cond)

    def credit_assigment(self, parents, offspring, op_idx):
        p1_vs_p2 = self.dominance_comparison(parents)
        p2_vs_p1 = self.dominance_comparison([parents[1], parents[0]])

        if(p1_vs_p2 or p2_vs_p1):
            if(p1_vs_p2): 
                non_dominated_parent = parents[0]
            else: 
                non_dominated_parent = parents[1]
            for i in range(2):
                individuals = [offspring[i], non_dominated_parent]
                ri_vs_p1 = self.dominance_comparison(individuals)
                if(ri_vs_p1):
                    self.n_penalty[op_idx] += 1
                else:
                    self.n_reward[op_idx] += 1
        else:
            for i in range(2):
                individuals = [offspring[i], parents[0]]
                ri_vs_p1 = self.dominance_comparison(individuals)
                individuals = [offspring[i], parents[1]]
                ri_vs_p2 = self.dominance_comparison(individuals)
                if((not ri_vs_p1) and (not ri_vs_p2)):
                    self.n_reward[op_idx] += 1
                else:
                    self.n_penalty[op_idx] += 1

    def non_dominating_curve_plotter(self):
        plt.figure(figsize=(6,5))
        PF = self.fast_non_dominated_sort(self.objective1_values, self.objective2_values)
        plt.xlabel('Solution size', fontsize=9)
        plt.ylabel('Classification error', fontsize=9)
        for idx in PF[0]:
            plt.scatter(self.objective2_values[idx], self.objective1_values[idx], c='red', marker="*", s=25)
        plt.show()

    def fit(self):
        # self.population = self.init_pop()
        RD = np.zeros((self.Q, self.LP))
        PN = np.zeros((self.Q, self.LP))
        self.n_reward = np.zeros(self.Q, dtype=int)
        self.n_penalty = np.zeros(self.Q, dtype=int)
        
        pops = []
        fitness_values_total = []
        # pops = np.array([]).reshape(0, self.d)
        for r_count in range(1):
            print(f"Run ({r_count})")
            self.population = self.init_pop()
            self.OSP = [1/self.Q for _ in range(self.Q)]
            self.generation_count = 0
            self.nFE = 0
            k = 0
            while self.nFE < self.max_nFE:
                self.generation_seed = np.random.randint(0, 4294967295, dtype=np.int64)
                self.objective1_values = [self.score_individual_1(individual) for individual in self.population]
                self.objective2_values = [self.score_individual_2(individual) for individual in self.population]
                PF = self.fast_non_dominated_sort(self.objective1_values, self.objective2_values)
                print('Best Individuals in Front-1 for Generation:', self.generation_count)
                for idx in PF[0]:
                    print(self.population[idx], end='')
                    print(f", O1: {self.score_individual_1(self.population[idx])} ", end='')
                    print(f"| O2: {self.score_individual_2(self.population[idx])}")
                print("\n")
                
                pop = np.array(self.population)
                fitness_values = self.evaluation(pop)
                index = np.arange(pop.shape[0]).astype(int)
                pareto_front_index = self.pareto_front_finding(fitness_values, index)
                # pop = pop[pareto_front_index, :]
                # fitness_values = fitness_values[pareto_front_index]
                print('Best Individuals in Front-1 for Generation:', self.generation_count)
                for idx in pareto_front_index:
                    print(pop[idx], end='')
                    print(f", O1: {fitness_values[idx, 0]} ", end='')
                    print(f"| O2: {fitness_values[idx, 1]}")
                print("\n")
                new_population = np.array(self.population[:])

                for i in range(int(self.pop_size/2)):
                    # Selecting operator using roulette wheel selection
                    operator_idx = np.random.choice(range(0, self.Q), p=self.OSP)
                    # Randomly select two individuals as parents
                    parents_idx = np.random.choice(range(0, self.pop_size), 2)
                    parents = [self.population[idx] for idx in parents_idx]
                    # Crossover
                    offspirng = self.crossover(parents, operator_idx)
                    # Mutation
                    mutated_offspring = [self.mutation(individual) for individual in offspirng]
                    self.credit_assigment(parents, offspirng, operator_idx)
                    
                    # np.append(new_population, mutated_offspring)
                    new_population = np.concatenate((new_population, mutated_offspring))
                    # new_population += mutated_offspring
                    self.nFE += 2
                    
                RD[k, :] = self.n_reward
                PN[k, :] = self.n_penalty
                self.n_reward = np.zeros(self.Q, dtype=int)
                self.n_penalty = np.zeros(self.Q, dtype=int)
                k += 1
                if(k == self.LP):
                    self.update_OSP(RD, PN)
                    k = 0 

                objective1_values = [self.score_individual_1(individual) for individual in new_population]
                objective2_values = [self.score_individual_2(individual) for individual in new_population]
                non_dominated_sorted_solutions = self.fast_non_dominated_sort(objective1_values[:], objective2_values[:])
                crowding_distance_values2=[]
                for i in range(len(non_dominated_sorted_solutions)):
                    crowding_distance_values2.append(self.crowding_distance(objective1_values[:], objective2_values[:], \
                                                                            non_dominated_sorted_solutions[i][:]))
                new_solution= []
                for i in range(len(non_dominated_sorted_solutions)):
                    non_dominated_sorted_solution2_1 = [self.index_locator(non_dominated_sorted_solutions[i][j],\
                                                                           non_dominated_sorted_solutions[i] ) \
                                                                            for j in range(0, len(non_dominated_sorted_solutions[i]))]
                    front22 = self.sort_by_values(non_dominated_sorted_solution2_1[:], crowding_distance_values2[i][:])
                    front = [non_dominated_sorted_solutions[i][front22[j]] for j in range(0, len(non_dominated_sorted_solutions[i]))]
                    front.reverse()
                    for value in front:
                        new_solution.append(value)
                        if(len(new_solution) == self.pop_size):
                            break
                    if (len(new_solution) == self.pop_size):
                        break
                self.population = [new_population[i] for i in new_solution]
                
                self.generation_count += 1


: 

In [ ]:
mobga_ds2 = MOBGA_AOS(max_nFE = 3000)
mobga_ds2.load_data('DS02.csv')
mobga_ds2.fit()

: 

In [ ]:
mobga_ds2.non_dominating_curve_plotter()

: 

In [ ]:
mobga_ds3 = MOBGA_AOS(max_nFE = 4000)
mobga_ds3.load_data('DS03.csv')
mobga_ds3.fit()

: 

In [ ]:
mobga_ds3.non_dominating_curve_plotter()

: 